Check whether it replaces existing data
Make this more airtight

In [24]:
import numpy as np
import faiss
import psycopg2

# FAISS Index - Use Inner Product for Cosine Similarity
dimension = all_embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)  
faiss.normalize_L2(all_embeddings)  # Normalize for cosine similarity
index.add(all_embeddings)

# Retrieve FAISS nearest neighbors
k = 10  # Top-10
distances, neighbors = index.search(all_embeddings, k + 1)  # +1 to exclude self-match

# Prevent self-matches
similarity_data = []
for i in range(len(db_ids)):  
    source_id = db_ids[i]

    for j in range(1, k + 1):  # Skip first result (self-match)
        target_id = db_ids[neighbors[i][j]]
        similarity = float(1 - distances[i][j])  # Convert FAISS L2 to similarity
        closeness = float(1 / (distances[i][j] + 1e-6))  # Prevent division by zero

        similarity_data.append((source_id, target_id, similarity, closeness))


In [21]:
import random

# **Get a few random entries to check their similar results**
sample_indices = random.sample(range(len(nodes)), 5)  # Select 5 random entries

print("\n🔗 **Sample Similar Sections**")
for idx in sample_indices:
    entry = nodes[idx]
    title, section = entry["title"], entry["section"]

    print(f"\n📜 **{title} — {section}**\n   🔍 Closest matches:")

    for rank, neighbor_idx in enumerate(neighbors[idx][1:4], start=1):  # Skip self-match
        neighbor = nodes[neighbor_idx]
        print(f"   {rank}. **{neighbor['title']}** — {neighbor['section']}")

    print("-" * 80)



🔗 **Sample Similar Sections**

📜 **logical form — 5. Descriptions and Analysis**
   🔍 Closest matches:
   1. **logical form** — 5. Descriptions and Analysis
   2. **music\, philosophy of** — 1.2 The Definition of “Music”
   3. **Twardowski\, Kazimierz** — 3.1 On the so-called relative truths (1900)
--------------------------------------------------------------------------------

📜 **God\, arguments for the existence of cosmological argument: teleological — see teleology: teleological arguments for God's existence — 1. Introduction**
   🔍 Closest matches:
   1. **God\, arguments for the existence of cosmological argument: teleological — see teleology: teleological arguments for God's existence** — 1. Introduction
   2. **epistemology: virtue** — 10. Expanding Horizons
   3. **impossible worlds** — 6.6 Compositionality
--------------------------------------------------------------------------------

📜 **federalism — 4.3 Sources of Stability**
   🔍 Closest matches:
   1. **intuitionistic

In [22]:

# **Bulk insert into PostgreSQL**
cur.executemany(
    "INSERT INTO sep_similarity (source_id, target_id, similarity, closeness_score) VALUES (%s, %s, %s, %s);",
    similarity_data
)
conn.commit()

cur.close()
conn.close()

print(f"✅ FAISS similarity graph stored in PostgreSQL! {len(similarity_data)} relationships added.")


✅ FAISS similarity graph stored in PostgreSQL! 349380 relationships added.
